# Páté cvičení - word2vec
Pro toto cvičení využijte již předtrénované GloVe (50) vektory.

Vektory lze stáhnout [zde (GloVe link)](https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip)

V archivu se nachází několik variant o různých dimenzích. Využijte dimenzi vektorů 50.

## Příprava dat

- Načtěte data z txt souboru (vždy je na řádku slovo a jeho vektor)
- Slova si uložete do matice o rozměrech n x 50 (n = počet slov)
- Vytvořte slovník, který zjistí index pro každé slovo (např. word2idx["slovo"])

In [1]:
import numpy as np
from numpy.linalg import norm

In [4]:
! mkdir glove
! wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip

mkdir: cannot create directory ‘glove’: File exists
--2025-03-18 10:05:56--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip
Resolving huggingface.co (huggingface.co)... 2600:9000:2127:e000:17:b174:6d00:93a1, 2600:9000:2127:ac00:17:b174:6d00:93a1, 2600:9000:2127:da00:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:2127:e000:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/stanfordnlp/glove/6471382cdd837544bf3ac72497a38715e845897d265b2b424b4761832009c837?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27glove.6B.zip%3B+filename%3D%22glove.6B.zip%22%3B&response-content-type=application%2Fzip&Expires=1742292357&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjI5MjM1N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zdGFuZm9yZG5scC9nbG92ZS82NDcxMzgyY2RkODM3NTQ0YmYzYWM3MjQ5N2EzODcxNWU4NDU4OTdkMjY1YjJiNDI0YjQ3NjE4MzIwM

In [2]:
! unzip glove.6B.zip glove.6B.50d.txt

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
caution: filename not matched:  -X


In [51]:
# Načtení dat ze souborů
with open("glove.6B.50d.txt", "r", encoding="utf-8") as f:
    vectors = f.read()
    vectors = vectors.splitlines()

In [52]:
# Uložení každého slova do slovníku a jeho příslušného vektoru
word2idx = dict()
idx2word = list()
matrix = np.zeros((len(vectors), 50))

idx = 0
for v in vectors:
    data : list = v.split(" ")
    word : str = data[0]
    vector : list = [float(i) for i in data[1:]]
    if(len(vector) > 1):
        word2idx[word] = idx
        idx2word += [word]
        matrix[idx,0:] = vector
        idx += 1
  

## Kosinová podobnost
Vytvořte funkci, která bude vracit kosinovou podobnost dvou vektorů
<br>
<br>
$
similarity(a,b) = cos(\theta) = \frac{a \cdot b}{\lVert a \lVert \lVert b \lVert}
$

In [53]:
def cossim(A,B):
    return A @ B / norm(A, axis=-1) / norm(B, axis=0)

In [54]:
prague_idx = word2idx["prague"]
czech_idx = word2idx["czech"]
similarity = cossim(matrix[prague_idx], matrix[czech_idx])
print(similarity)

# Očekávané výsledky
# similarity = 0.6934045343089018

0.6934045343089016


## Nalezení podobných slov
Vytvořte funkci, která vrátí 10 nejpodobnějších slov.

In [68]:
def most_similar(A):
    A = matrix[word2idx[A]]
    B = matrix.T
    sims = cossim(A, B)
    idx = np.argsort(sims)[::-1]
    return {
        idx2word[i]: sims[i]
        for i in idx[1:11]
    }

In [69]:
most_similar("terrible")

{'horrible': 0.9373438294745688,
 'awful': 0.8872899998451528,
 'tragic': 0.8339797859732645,
 'dreadful': 0.8279697448704044,
 'tragedy': 0.8180269496589326,
 'horrific': 0.8108952997029031,
 'nightmare': 0.7994893093099843,
 'unfortunate': 0.7938124514408333,
 'horrendous': 0.7856658154073574,
 'unfortunately': 0.7838044061147325}

Očekávaný výsledek:

    [('horrible', 0.9373438294745691),
     ('awful', 0.8872899998451529),
     ('tragic', 0.8339797859732649),
     ('dreadful', 0.8279697448704045),
     ('tragedy', 0.8180269496589323),
     ('horrific', 0.810895299702903),
     ('nightmare', 0.7994893093099841),
     ('unfortunate', 0.7938124514408333),
     ('horrendous', 0.7856658154073575),
     ('unfortunately', 0.7838044061147322)]


## Sčítání a odčítání vektorů a jejich využití

Jak se vektory slov dají sčítat a odčítat naleznete [zde](https://medium.com/analytics-vidhya/ideas-for-using-word2vec-in-human-learning-tasks-1c5dabbeb72e)

In [70]:
# Pomocí sčítání a odčítání vektorů nalezněte chybějící slovo ve výrazu “Berlin is to Germany as … is to Italy”
def most_similar_2(A):
    B = matrix.T
    sims = cossim(A, B)
    idx = np.argsort(sims)[::-1]
    return {
        idx2word[i]: sims[i]
        for i in idx[:10]
    }

berlin = matrix[word2idx['berlin']]
germany = matrix[word2idx['germany']]
italy = matrix[word2idx['italy']]
most_similar_2(berlin - germany + italy)
# Očekávaný výsledek: ('rome')

{'rome': 0.852226793994773,
 'venice': 0.8517536858634684,
 'italy': 0.7876652944646821,
 'naples': 0.767179898217344,
 'lisbon': 0.7613592589119934,
 'turin': 0.7429368578810202,
 'genoa': 0.7340990394824866,
 'bologna': 0.7277822010059736,
 'italian': 0.7189798949114362,
 'milan': 0.7114141133788309}